In [18]:
import torch
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from LoadDataset.utils import (
    MaskDetectionDataSet,
    dataset_split,
    dataloader
)
from Model.ResNet import MaskModel


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [20]:
torch.manual_seed(100)

In [21]:
data_dir = "../data"

In [22]:
dataset = MaskDetectionDataSet(data_dir)

In [23]:
dataset.classes

['with_mask', 'without_mask']

In [24]:
train_ds, valid_ds = dataset_split(dataset, val_ratio = 0.2)

In [25]:
train_dl = dataloader(train_ds, batch_size=32, shuffle=True)
valid_dl = dataloader(valid_ds, batch_size=32, shuffle=False)

In [26]:
for img, labels in train_dl:
    print(img.shape)
    print(labels.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


Transfer Learning using pre-trained ResNet34 model

In [28]:
model = MaskModel(len(dataset.classes), pretrained=False).to(device)

In [29]:
summary(model.to(device), input_size=(3, 224, 224), batch_size = 32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
            Conv2d-5           [32, 64, 56, 56]          36,864
       BatchNorm2d-6           [32, 64, 56, 56]             128
              ReLU-7           [32, 64, 56, 56]               0
            Conv2d-8           [32, 64, 56, 56]          36,864
       BatchNorm2d-9           [32, 64, 56, 56]             128
             ReLU-10           [32, 64, 56, 56]               0
       BasicBlock-11           [32, 64, 56, 56]               0
           Conv2d-12           [32, 64, 56, 56]          36,864
      BatchNorm2d-13           [32, 64, 56, 56]             128
             ReLU-14           [32, 64,

Optimizer

In [30]:
optimizer = Adam(model.parameters())

Loss function

cross entroy loss

In [31]:
loss_fn = nn.CrossEntropyLoss()

Accuracy

In [32]:
def accuracy(y_true, y_pred):
    return torch.tensor(torch.sum(y_true==y_pred).item()/len(y_pred))

In [33]:
model.train()
training_loss = []
validation_loss = []
training_accuracy = []
validation_accuracy = []
epoch = 2
for i in range(epoch):
    train_loss = []
    val_loss = []
    train_accuracy = []
    val_accuracy = []
    train_loop = tqdm(train_dl, leave=True)
    for x, labels in train_loop:
        train_loop.set_description(f"Epoch {i+1}")
        optimizer.zero_grad()
        y = model(x)
        y = F.softmax(y)
        loss = loss_fn(y.float(), labels)
        loss.backward()
        optimizer.step()

        _, pred =torch.max(y, dim = 1)
        accuracy_val = accuracy(labels, pred)
        train_loss.append(loss.item())
        train_accuracy.append(accuracy_val.item())

        train_loop.set_postfix(
            train_loss=sum(train_loss) / len(train_loss),
            train_accuracy=sum(train_accuracy) / len(train_accuracy),
        )

    val_loop = tqdm(valid_dl, leave=True)
    with torch.no_grad():
        for x, labels in val_loop:
            y = model(x)
            y = F.softmax(y)
            loss = loss_fn(y.float(), labels)

            _, pred =torch.max(y, dim = 1)
            accuracy_val = accuracy(labels, pred)
            val_loss.append(loss.item())
            val_accuracy.append(accuracy_val.item())

            val_loop.set_postfix(
                train_loss=sum(train_loss) / len(train_loss),
                train_accuracy=sum(train_accuracy) / len(train_accuracy),
                val_loss=sum(val_loss) / len(val_loss),
                val_accuracy=sum(val_accuracy) / len(val_accuracy),
            )

    training_loss.append(sum(train_loss) / len(train_loss))
    training_accuracy.append(sum(train_accuracy) / len(train_accuracy))
    validation_loss.append(sum(val_loss) / len(val_loss))
    validation_accuracy.append(sum(val_accuracy) / len(val_accuracy))

100%|██████████| 48/48 [01:09<00:00,  1.45s/it, train_accuracy=0.879, train_loss=0.43, val_accuracy=0.898, val_loss=0.412]


In [34]:
torch.save(model, "../model.pt")